In [ ]:
!pip install sentencepiece
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar -xzf ldcc-20140209.tar.gz

--2025-11-08 09:07:59--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  4.70MB/s    in 1.8s    

2025-11-08 09:08:01 (4.70 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
# ==========================================
# SentencePiece (Unigram LM) 学習前後の比較
# ==========================================
import sentencepiece as spm
import os

In [ ]:
# -------------------------
# 1. 学習用コーパスを作成
# -------------------------
import json, glob

with open("ja_corpus.txt", "w", encoding="utf-8") as out:
    for file in glob.glob("text/*/*.txt"):
        with open(file, encoding="utf-8") as f:
            lines = f.readlines()[3:]  # 最初3行はヘッダ
            out.write("".join(lines) + "\n")

print("✅ Livedoorニュースコーパスを作成しました")

✅ Livedoorニュースコーパスを作成しました


In [ ]:
# -------------------------
# 2. 学習前（ベースライン）: 未学習状態
# -------------------------
def naive_tokenize(text):
    """文字単位で単純に分割する（学習前の基準）"""
    return list(text)

test_sentence1 = "私は自然言語処理を勉強しています。"
test_sentence2 = "そのソフトウェアは多くのユーザーから賛同を得ている。"
print("🔹 学習前（文字単位分割）:")
print(naive_tokenize(test_sentence1))
print()
print(naive_tokenize(test_sentence2))
print()


🔹 学習前（文字単位分割）:
['私', 'は', '自', '然', '言', '語', '処', '理', 'を', '勉', '強', 'し', 'て', 'い', 'ま', 'す', '。']

['そ', 'の', 'ソ', 'フ', 'ト', 'ウ', 'ェ', 'ア', 'は', '多', 'く', 'の', 'ユ', 'ー', 'ザ', 'ー', 'か', 'ら', '賛', '同', 'を', '得', 'て', 'い', 'る', '。']



In [ ]:
# -------------------------
# 3. SentencePiece モデル学習 (Unigram LM)
# -------------------------
if not os.path.exists("ja_spm.model"):
    spm.SentencePieceTrainer.train(
        input="ja_corpus.txt",
        model_prefix="ja_spm",
        vocab_size=2600,
        model_type="unigram",
        character_coverage=0.9995,
        pad_id=0, unk_id=1, bos_id=2, eos_id=3
    )
    print("✅ モデルを学習しました。")
else:
    print("✅ 既存モデルを使用します。")

✅ モデルを学習しました。


In [ ]:
# -------------------------
# 4. 学習後モデルのロード
# -------------------------
sp = spm.SentencePieceProcessor(model_file="ja_spm.model")

# -------------------------
# 5. 学習後の分割を比較
# -------------------------
tokens_after = sp.encode(test_sentence1, out_type=str)

print("🔹 学習後（SentencePiece 分割）:")
print(tokens_after)

tokens_after = sp.encode(test_sentence2, out_type=str)
print(tokens_after)

print()

🔹 学習後（SentencePiece 分割）:
['▁', '私', 'は', '自', '然', '言', '語', '処', '理', 'を', '勉', '強', 'して', 'い', 'ます', '。']
['▁', 'その', 'ソ', 'フ', 'ト', 'ウ', 'ェ', 'ア', 'は', '多', 'く', 'の', 'ユ', 'ー', 'ザ', 'ー', 'から', '賛', '同', 'を', '得', 'ている', '。']

